In [2]:
import nltk
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

df = pd.read_csv(r'C:\Users\gunja\OneDrive\Desktop\DEDA PROJECT\musktweets.csv')

df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gunja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gunja\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gunja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gunja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gunja\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Unnamed: 0,MESSAGE,date
0,0,"Please ignore prior tweets, as that was someon...",2010-06-04 18:31:57+00:00
1,1,Went to Iceland on Sat to ride bumper cars on ...,2011-12-01 09:55:11+00:00
2,2,I made the volume on the Model S http://ow.ly/...,2011-12-01 10:29:04+00:00
3,3,"Great Voltaire quote, arguably better than Twa...",2011-12-03 08:20:28+00:00
4,4,That was a total non sequitur btw\n26\n14\n50,2011-12-03 08:22:07+00:00


In [4]:
#data cleaning

df = df[['MESSAGE', 'date']]

df['MESSAGE'] = df['MESSAGE'].str.lower()

df['MESSAGE'] = df['MESSAGE'].apply(lambda x: re.sub(r"(http\S+|www\S+|@\S+)", "", str(x)))

df['MESSAGE'] = df['MESSAGE'].apply(lambda x: re.sub(r"[^a-zA-Z\s]", "", str(x)))

df['MESSAGE'] = df['MESSAGE'].apply(lambda x: nltk.word_tokenize(str(x)))

stop_words = set(stopwords.words('english'))
df['MESSAGE'] = df['MESSAGE'].apply(lambda x: [word for word in x if word not in stop_words])


def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
    return lemmatized_words

df['MESSAGE'] = df['MESSAGE'].apply(lambda x: lemmatize_text(x))

df['MESSAGE'] = df['MESSAGE'].apply(lambda x: ' '.join(x))

df.to_csv('cleaned_dataset.csv', index=False)

df.head()

,MESSAGE,date
0,please ignore prior tweet someone pretending a...,2010-06-04 18:31:57+00:00
1,went iceland sat ride bumper car ice country v...,2011-12-01 09:55:11+00:00
2,made volume model go need work miniature stone...,2011-12-01 10:29:04+00:00
3,great voltaire quote arguably better twain hea...,2011-12-03 08:20:28+00:00
4,total non sequitur btw,2011-12-03 08:22:07+00:00


In [5]:
# Perform sentiment analysis
df['sentiment_score'] = df['MESSAGE'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Classify tweets based on sentiment scores
df['sentiment'] = df['sentiment_score'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))

# Display the resulting DataFrame with sentiment labels
print(df.head())

df.to_csv('sentiment_dataset.csv', index=False)

                                             MESSAGE  \
0  please ignore prior tweet someone pretending a...   
1  went iceland sat ride bumper car ice country v...   
2  made volume model go need work miniature stone...   
3  great voltaire quote arguably better twain hea...   
4                             total non sequitur btw   

                        date  sentiment_score sentiment  
0  2010-06-04 18:31:57+00:00             0.00   Neutral  
1  2011-12-01 09:55:11+00:00             0.65  Positive  
2  2011-12-01 10:29:04+00:00             0.00   Neutral  
3  2011-12-03 08:20:28+00:00             0.55  Positive  
4  2011-12-03 08:22:07+00:00             0.00   Neutral  
